In [ ]:
import src.lol_utils as lol
import pandas as pd

## Loading Data
Loads the dataframe from SQL, and sets it equivalent to df. In doing so, we also parse it so the dataframe is structured correctly (in Python lists) to avoid formatting problems.


In [ ]:
df = lol.fetch_from_sql(schema='soloq', table_name='lol_analytics')

## Preprocessing Data
After the data has been loaded, we clean the data up, detecting bad rows, duplicates, and determining if there are any champions that in the set of champions.

After detecting the bad rows, we delete them and create a new cleaned dataframe with only the good rows (any rows that do not have duplicates, are not in the consistent format of 5 champs per team, and with champions that are recognized in the champion list).

In [ ]:
from src.lol_utils import champion_list

# perfect we have the data! now let's do some pre-processing

# detect bad rows:
bad_nulls = df[(df['blue_team'].apply(len) == 0) | (df['red_team'].apply(len) == 0)]
bad_counts = df[(df['blue_team'].apply(len) != 5) | (df['red_team'].apply(len) != 5)]
# duplicates
df['blue_dup'] = df['blue_team'].apply(lambda L: len(set(L)) != len(L))
df['red_dup']  = df['red_team'].apply(lambda L: len(set(L)) != len(L))

# unknown champs: compare against canonical set
canonical = set(champion_list)  # fill with your champion list
df['blue_unknown'] = df['blue_team'].apply(lambda L: [c for c in L if c not in canonical])
df['red_unknown']  = df['red_team'].apply(lambda L: [c for c in L if c not in canonical])

In [ ]:
# clean up bad rows where blue_unknown or red_unknown is non-empty, or blue_dup/red_dup is True, or team_parsed length != 5
clean_df = df[
    (df['blue_dup'] == False) &
    (df['red_dup'] == False) &
    (df['blue_unknown'].apply(len) == 0) &
    (df['red_unknown'].apply(len) == 0) &
    (df['blue_team'].apply(len) == 5) &
    (df['red_team'].apply(len) == 5)
].copy()

clean_df['target'] = (clean_df['winner'] == 'blue').astype(int)

df = clean_df.copy()

## Feature Engineering

In order to turn these champions list into something useful for our machine learning model, we will create new columns for the model to interpret.

1. We create side-agnostic champion presence:
A binary column that indicated whether or not the champion was included in the game.

2. Team synergy score: A custom made formula that calculates the combinations of each ally champion with another ally champion's synergy, calculated using a elo-inspired formula. A column for each team will contain the team's synergy score indicating how synergistic the team is, with positive indiciating more synergy and negative indicating vice versa.

3. Counter-delta: A custom made formula that instead of calculating the combinations of each ally champion, calculates the combinations of a champion on one team vs every champion on the other team. The counter delta is calculated for each champion on each team, and the difference indicates how much one team counters another. Positive indicating blue team has a significance counter over red team, while negative shows the opposite. Stored inside a column for each game.

In [ ]:
# We'll use the cleaned DataFrame with parsed teams and target

# Champion win rates and synergy only
champ_winrates = lol.calculate_champion_winrates(df)
df = lol.add_synergy_and_counter_scores(df, champ_winrates)  # if you made this synergy-only, it’s fine

# Side-agnostic presence features
for champ in lol.champion_list:
    df[f'{champ}'] = df.apply(
        lambda row: int(champ in row['blue_team'] or champ in row['red_team']),
        axis=1
    )

# Build features (presence + synergy only)
champ_presence_cols = [f'{champ}' for champ in lol.champion_list]
extra_cols = ['synergy_blue', 'synergy_red', 'counter_delta']
feature_cols = champ_presence_cols + [c for c in extra_cols if c in df.columns]

# Ensure booleans -> ints
for col in feature_cols:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int)

## Model Matrix
We will then pick the feature columns to be created in our project, this includes of course, our feature engineered columns that we created. We will use this matrix with the combination of the target column to make predictions for the match.

In [ ]:
X = df[feature_cols]
y = df['target']

In [ ]:
df.to_pickle('cleaned_features.pkl')